In [4]:
CONFIG_PATH = "configs/debug_small_LM.yml"
# CONFIG_PATH = "configs/debug_med_ID_sim_new_votes.yml"

In [5]:
from collections import Counter, defaultdict
import os
import random
import shutil
import numpy as np
import pandas as pd
from superdebug import debug
import torch
from data import get_model_input
from utils import get_config, join_sets, load_model, print_log, save_model, load_model_dict
from model import get_best_model

In [6]:
config = get_config(CONFIG_PATH, "_curation")
all_feature_columns, target, train_model_input, test_model_input, feature_names, original_feature_map, max_voted_users, train_data, test_data, test_data_info = get_model_input(config)
model, user_embedding = get_best_model(config, all_feature_columns, feature_names)

GPU ready...
Smart using cuda:2
------------------ 2022-07-21 06:53:03 ------------------
DEBUG: at /home/TableSense/largedisk/wanrong/Curation-Modeling/data.py:228 get_model_input
Loading prepared data...
------------------ 2022-07-21 06:53:03 ------------------


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
test_len = len(test_model_input["USERNAME"])
# test_submissions = set(test_model_input["SUBMISSION_ID"])
test_data = pd.DataFrame(test_model_input) 



,USERNAME,SUBREDDIT,AUTHOR,CREATED_TIME,NSFW,UPVOTED_USERS,DOWNVOTED_USERS,input_ids,token_type_ids,attention_mask
1720,927,16,156,0.935177,0.0,"[112, 1506, 437, 1238]","[2052, 1028, 1542, 1544, 2056, 2060, 526, 1553...","(tensor(101), tensor(1996), tensor(2880), tens...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
748,166,50,48,0.996082,0.0,"[1536, 1378, 739, 1734, 1644, 589, 46, 688, 49...","[418, 231, 650, 1228, 1709, 908, 245, 1369, 18...","(tensor(101), tensor(1005), tensor(1996), tens...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
3915,724,38,39,0.975998,0.0,"[480, 36, 2184, 331, 925, 1393, 114, 1977, 860...","[1411, 1476, 777, 1931, 1036, 2256, 466, 1874,...","(tensor(101), tensor(2445), tensor(2008), tens...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
3950,1602,22,132,0.993641,0.0,"[1796, 1480, 1865, 10, 981, 341, 1372, 2018, 6...","[162, 2023, 2216, 2120, 1163, 1486, 825, 55, 2...","(tensor(101), tensor(2412), tensor(2144), tens...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
5942,64,34,109,0.992643,0.0,"[204, 1293, 237, 1392, 1808, 1943, 126]","[1155, 772, 901, 646, 1671, 776, 1287, 264, 20...","(tensor(101), tensor(26111), tensor(16215), te...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
...,...,...,...,...,...,...,...,...,...,...
3838,1602,9,50,0.993091,0.0,"[636, 521, 1485, 333, 656, 2258, 1043, 213, 20...","[192, 576, 960, 1345, 2183, 72, 1098, 1746, 16...","(tensor(101), tensor(15697), tensor(2085), ten...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
221,301,51,87,0.992553,1.0,"[1762, 1285, 2054, 200, 1900, 173, 1741, 1935,...","[1792, 3, 1287, 265, 10, 783, 795, 2080, 35, 2...","(tensor(101), tensor(2039), tensor(22994), ten...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
267,1194,51,87,0.992553,1.0,"[1762, 1285, 2054, 200, 1900, 173, 1741, 1935,...","[1792, 3, 1287, 265, 10, 783, 795, 2080, 35, 2...","(tensor(101), tensor(2039), tensor(22994), ten...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
107,577,50,105,0.990259,0.0,"[1224, 41, 2220, 1999, 181, 375, 1369]","[2240, 612, 2020, 806, 1192, 2155, 1260, 2158,...","(tensor(101), tensor(13300), tensor(16558), te...","(tensor(0), tensor(0), tensor(0), tensor(0), t...","(tensor(1), tensor(1), tensor(1), tensor(1), t..."
